In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train=pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test=pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
sub=pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

In [3]:
train

In [4]:
test

In [5]:
sub

In [6]:
train.dtypes  #train의 dtype  ,  object는 계산 불가

In [7]:
train1=train.drop(['datetime','casual', 'registered','count'],axis=1)
test1=test.drop(['datetime'],axis=1)
target=train['count']

**RandomForestRegressor**

In [8]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train1, target)   #randomforest 알고리즘으로 fitting

In [9]:
 res_rf=rf.predict(test1)  #예측

In [10]:
sub['count']=res_rf  #예측된 결과를 sub의 count에 넣는다.

In [11]:
sub.to_csv('bikerf.csv',index=False) #파일로 만든다.

# **여기까지 base code**

# **1. data 분석**

In [12]:
train['count'].mean()

In [13]:
train.describe()

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
sns.histplot(train['count'])  #log scalen 

In [16]:
train[train['count']==1] #count가 1인 것들을 출력

In [17]:
train[train['count']==977] #count가 977인 것은 max 1개 밖에 없다

In [18]:
train[train['count']==900] #count가 900인

# **2. 시간 데이터 활용**

In [19]:
train1=train.copy()  #따로 쓰려고 캎피
test1=test.copy()

In [20]:
train1['datetime']=train1['datetime'].astype('datetime64')
test1['datetime']=test1['datetime'].astype('datetime64')

In [21]:
train1['year']=train1['datetime'].dt.year
train1['month']=train1['datetime'].dt.month
train1['day']=train1['datetime'].dt.day
train1['hour']=train1['datetime'].dt.hour
train1['weekday']=train1['datetime'].dt.weekday

In [22]:
test1['year']=test1['datetime'].dt.year
test1['month']=test1['datetime'].dt.month
test1['day']=test1['datetime'].dt.day
test1['hour']=test1['datetime'].dt.hour
test1['weekday']=test1['datetime'].dt.weekday

In [23]:
train2=train1.drop(['datetime','casual', 'registered','count'],axis=1)
test2=test1.drop('datetime',axis=1)

In [24]:
rf1=RandomForestRegressor()
rf1.fit(train2,target)

In [25]:
res_rf1=rf1.predict(test2)

In [26]:
sub['count']=res_rf1
sub.to_csv('bike_rf1.csv',index=False)

# **모델 향상**

In [27]:
from lightgbm import LGBMRegressor
lg=LGBMRegressor()
lg.fit(train2, target)

In [28]:
res_lg=lg.predict(test2)
sub['count']=res_lg
sub.to_csv('bike_lg.csv', index=False)

In [29]:
lg2=LGBMRegressor(n_estimators=1000,learning_rate=0.01,colsample_bytree=0.8)
lg2.fit(train2, target)
res_lg2=lg2.predict(test2)
sub['count']=res_lg2
sub.to_csv('bike_lg2.csv',index=False)

# **3.log scale 적용**

In [30]:
sns.histplot(train['count']) #log scale

In [31]:
sns.histplot(np.log(train['count']))

In [32]:
lg3=LGBMRegressor(n_estimators=1000,learning_rate=0.01,colsample_bytree=0.8)
lg3.fit(train2, np.log(target))
res_lg3=lg3.predict(test2)
sub['count']=res_lg3
sub.to_csv('submit_bike_lg3.csv',index=False)

# rf와 catboost에서도 적용

# **XGBoost**

In [33]:
from xgboost import XGBRegressor
ixg=XGBRegressor(n_estimators=1000,learning_rate=0.01,colsample_bytree=0.8)
ixg.fit(train2,np.log(target))
res_ixg=ixg.predict(test2)
sub['count']=res_ixg
sub.to_csv('submit_bike_ixg.csv',index=False)

# **catboost**

In [34]:
from catboost import CatBoostRegressor
icat=CatBoostRegressor(n_estimators=1000,learning_rate=0.01)
icat.fit(train2, np.log(target))
res_icat=icat.predict(test2)
sub['count']=res_icat
sub.to_csv('submit_bike_icat.csv',index=False)

# **randomforest**

In [35]:
rf1=RandomForestRegressor()
rf1.fit(train2,np.log(target))
res_rf1=rf1.predict(test2)
sub['count']=res_rf1
sub.to_csv('submit_bike_rf1.csv',index=False)